# Data Cleaning

## Duplicates

Let's start by removing any data duplicates that add nothing to the dataset. We should compare the number of rows before and after removing the duplicates.

In [ ]:
#imports and reading dataset
import pandas as pd
import sklearn as skl
import nltk
import numpy as np

fake = pd.read_csv("./archive/fake.csv", na_values=[""])

In [ ]:
#cleaning duplicates
print("Number of rows before: ", len(fake.index))

fake = fake.drop_duplicates()

print("Number of rows after: ", len(fake.index))


So we can see that there are no duplicate rows. Moving on to missing data.

## Missing data

We should find all the rows with missing data and acknowledge every missing information column-wise. Therefore, we should see check for each column the missing information and see if we should either remove the collumn totally or find a viable substitution for each column.

In [ ]:
print(fake.isnull().sum().sort_values(ascending=False))


Here we can see a descending order of the number of missing values per collumn. We will now analyse each column and study if it is worth "fixing" or substituting the missing values or just delete the column all together.

As we can see, we have a column named "main_img_url" that doesn't provide any useful data for the study of this dataset. Because of that, we decided to remove it all together.

In [ ]:
fake.drop(['main_img_url'], axis=1, inplace=True)

In the column "author" there are two different cases that caught our attention: there are "anonymous" authors and just missing authors. Since both of these cases are comparable, because there is no info about the author in neither of them, we decided to make them all the same and add "anonymous" to the rows where the "author" info is missing.

In [ ]:
fake["author"]=fake["author"].fillna("Anonymous")
#fake.to_csv("fake_2.csv", index=False)

As we can also see, there are 12 rows without a thread title. This doesn't allow us group up the fake news into threads because we don't know where they belong. Since it is such a small ammount of news (12) we decided that they should removed as they don't represent a big sample.

In [ ]:
print("Number of rows before: ", len(fake.index))

fake = fake[fake['thread_title'].notna()]

print("Number of rows after: ", len(fake.index))


Since we are going to evaluate and perform text search based on the text of the news, news with no text become irrelevant to this dataset. Therefore, we are going to remove those rows as well.

In [ ]:
print("Number of rows before: ", len(fake.index))

fake = fake[fake['text'].notna()]

print("Number of rows after: ", len(fake.index))


There are news with no title which we have found to be in the same thread where only the news in first place (order_in_thread=0) has got a title. Let's verify if all the existing titles are the same as the thread_title associated.

In [ ]:
# get unique values from each column
titles = fake['title'].unique()
thread_titles = fake['thread_title'].unique()

#check if size is the same
print("Number of unique titles:", len(titles))
print("Number of unique thread_titles:", len(thread_titles),"\n")

# convert every value to string
for i in range(0, len(titles)):
    titles[i]=str(titles[i])

for i in range(0, len(thread_titles)):
    thread_titles[i] = str(thread_titles[i])

#sort arrays
np.sort(titles)
np.sort(thread_titles)


# print every different title - compare to bigger list: thread_titles
for element in thread_titles:
    if(element not in titles):
        print("Different thread_title:", element)



As we can see, there are different titles and thread_titles, making it impossible for us to eliminate one of those columns. We can, however, substitute the data and eliminate rows which don't have any of these informations, as it is impossible for us to track the news associated to any theme or key word.

In [ ]:
print("Number of rows with no title and no thread_title:", len(fake.loc[fake['title'].isna() & fake['thread_title'].isna()]))


Since there are no rows with neither of the values in the referred columns, we should turn our attention to how to correct the missing data in these columns. For every row with no title, we will substitute the value with the thread_title instead, since the thread_title will be in some way connected to the theme of the news.

# TALK TO CLARA: i put the thread_titles in the missing titles, just to not have any missing data anywhere

In [ ]:
fake.title.fillna(fake.thread_title, inplace=True)


Moreover, there are news with no 'country' value associated, more specifically, 176 news. To make it easier to analyze this data and not have missing values, we will drop these rows, since we can't track the origin of the news and it is a rather small ammount of data that is discarded (176 rows), given the size of the dataset.

In [ ]:
print("Number of rows before: ", len(fake.index))

fake = fake[fake['country'].notna()]

print("Number of rows after: ", len(fake.index))


As domain rank is a metric, going from 0 to 100, the later being the strongest, evaluated based on many factors dependent on user searches and the authority of a certain domain, if a domain doesn't have an available ranking, we can suppose it is 0 since it hasn't been evaluated yet. That can be due to lack of information or visits to that domain. That being said, we will substitute every missing value in the column "domain_rank" by 0.

In [ ]:
fake["domain_rank"] = fake["domain_rank"].fillna(0)

Let's take a new look at the null values per column and see the final dataset, with no missing data.

In [ ]:
print(fake.isnull().sum().sort_values(ascending=False))

## Broken Data & Data Types

As in all datasets, there are rows with broken data, as in, data that was unsuccessfully crawled and filled in rows with broken values. Let's find these rows in our dataset and correct or drop them.

The first case we noticed were some rows filled with 0 and 1, namely in the title and text columns. Let's locate them and print them, to analize the data and see if it should be dropped. If we can't find any and since we already cleaned some of the data, it means they were already dropped in the steps before.


In [ ]:
print(fake.loc[fake['title'] == "0"])
print(fake.loc[fake['title'] == "1"])

print(fake.loc[fake['text'] == "0"])
print(fake.loc[fake['text'] == "1"])


As an additional verification that the above statement was correct, we opened the dataset in excel and verified that every case of broken data was indeed gone from the dataset at this point of the cleaning.

The other big case for broken data that we found were some news that weren't correctly crawled and just extended the text of the news throughout several columns. There are several ways of correcting this issue, but in order to get all the cases and clean the data the most, we will start by verifying column by column if the data types and formats are correct, dropping the columns in which that isn't verified.

Let's start with the column uuid, which should have both letters and numbers. If any row doesn't have both, we will drop it. This already removes most of the cases mentioned before.

In [ ]:
print("Number of rows before: ", len(fake.index))

# fake = fake[fake.uuid.str.regex('^(?=.*\b(?=\S*[a-zA-Z])(?=\S*[0-9]))')]

# fake = fake['uuid'].str.extract('(?=.*\b(?=\S*[a-zA-Z])(?=\S*[0-9]))')

# fake = fake[~fake['uuid'].str.isalpha()]

fake['uuid'].filter(regex=r'[A-Za-z\s]')  #TODO: não apaga nada? já tentei mil formas diferentes. é suposto filtrar tudo o que tivesse apenas texto e espaços

print("Number of rows after: ", len(fake.index))


Moving on to the next column, ord_in_thread. It must be a number, therefore let's eliminate every row that doesn't just contain a number. If there is a row with alpha characters in it, the type of the column won't be int64 so let's first verify the type and eliminate non numeric rows if it isn't 'int64'.

In [ ]:
print(fake['ord_in_thread'].dtypes)

As it returns 'int64' we are sure that the whole column only contains numeric values. Moving on to 'author', there are 2 errors to fix: wrong datatypes and the "-NO AUTHOR-" rows. As it is very hard to verify the correctness of the author name, as it is very hard to define "correctness" in these cases, and not exclude valid names with numeric digits using the usual rules for identifying names (since authors can have numbers in their username), we will move on to correcting the "-NO AUTHOR-" cases, where it should be replaced by "Anonymous", as we did before with null values.

In [ ]:
fake["author"] = fake["author"].replace("-NO AUTHOR-","Anonymous")

Next up we have the column "published", that should only contain dates. We can verify this through a regex expression that verifies the format of the dates, as well as the datatypes present.

In [ ]:
print("Number of rows before: ", len(fake.index))

# fake = fake[fake['published'].str.extract(r'(\d{4})-\d{2}-\d{2}')] #dates - FIXME: extract somehow makes all the columns NaN
fake=fake[fake['published'].str.contains('\d{4}-\d{2}-\d{2}')]

print("Number of rows after: ", len(fake.index))


The next column to verify is the column "title". We have cleaned this column before so some work was already done. We should just verify that the titles contain uppercase and lowercase letters, numbers, punctuation and whitespaces.

In [ ]:
print("Number of rows before: ", len(fake.index))

# fake = fake[fake['title'].str.extract(r'[A-Za-z0-9!.,-?:\s]')] #TODO

print("Number of rows after: ", len(fake.index))


Moving on to the "text" column, we must do the same verification as in the "title" column.

In [ ]:
print("Number of rows before: ", len(fake.index))

# fake = fake[fake['text'].str.extract(r'[A-Za-z0-9!.,-?:\s]')] #TODO

print("Number of rows after: ", len(fake.index))


Next up we have the "language" column. This column must only contain one word and no digits, so the funtion isalpha() will do the job.

In [ ]:
print("Number of rows before: ", len(fake.index))

fake['language'] = fake['language'].apply(str)
fake = fake[fake.language.str.isalpha()] #TODO: is this working? i think so

print("Number of rows after: ", len(fake.index))


Next up is the column "crawled", which should only contain the date in which the data was crawled.

In [ ]:
print("Number of rows before: ", len(fake.index))

fake = fake[fake['crawled'].str.contains(r'(\d{4})-\d{2}-\d{2}')]  # dates

print("Number of rows after: ", len(fake.index))


# TODO: erase text rows with whitespace only. keep cleaning after crawled

And now, since the data cleaning is finished, we can save the new clean data to a new csv file, to be used later.

In [ ]:
fake.to_csv("fake_clean.csv", index=False)